In [ ]:
base_dir="/mnt/efs/shared/meg_shared_scripts/meg-kb"
# data_ac="indeeda-meg-ac"
# data_pt="indeeda-meg-pt"
yutong_base_dir="/home/ubuntu/users/yutong"

In [2]:
%cd $base_dir/src/concept_learning/

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/concept_learning


In [3]:
from tqdm.notebook import tqdm
import argparse
import re
import numpy as np
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr, entropy, gmean
import random
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import BertTokenizer, BertModel, BertForMaskedLM
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import json
from collections import defaultdict
import time

import logging
from sklearn.cluster import KMeans, AgglomerativeClustering
import pandas as pd
import os
import sys
import math
from annoy import AnnoyIndex

from spacy.lang.en import English
nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer'))
spacy_tokenizer = nlp.tokenizer

from compute_concept_clusters import knn
from compute_keyphrase_embeddings import ensure_tensor_on_device, mean_pooling

from utils import LMProbe, LMProbe_GPT2
from utils import load_embeddings, load_seed_aligned_concepts, load_seed_aligned_relations, load_benchmark
from utils import get_masked_contexts

from roberta_ses.interface import Roberta_SES_Entailment

In [4]:
seed_aligned_concepts_path = os.path.join(base_dir, f'data/indeed-benchmark/seed_aligned_concepts.csv')
seed_aligned_relations_path = os.path.join(base_dir, f'data/indeed-benchmark/seed_aligned_relations.csv')

# Data Preprocessing

In [2]:
# Input: text corpus
# step 1: extract key phrases (autophrase)
# step 2: generate embeddings

## Extract Key Phrases

In [5]:
# details here: https://github.com/rit-git/meg-kb/tree/main/src/keyword_extraction

In [6]:
#change to keyword extractor directory
%cd $base_dir/src/keyword_extraction/

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/keyword_extraction


In [7]:
!chmod +x ./corpusProcess.sh

In [10]:
# select the dataset and thread no
data_ac = 'indeeda-meg-ac'
data_pt = 'indeeda-meg-pt'
thread = 1

In [11]:
# process corpus and generate key prhases (long time! ~90min)
!./corpusProcess.sh $data_ac $thread

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/keyword_extraction
===Corpus Name: indeeda-meg-ac===
===Current Path: /mnt/efs/shared/meg_shared_scripts/meg-kb/src/keyword_extraction===
===Cleaning input corpus===
===Running AutoPhrase===
make: Nothing to be done for 'all'.
===RAW_TRAIN: ../../../data/indeeda-meg-ac/source/corpus.clean.txt===
auto_phrase.sh parameters: indeeda-meg-ac ../../../data/indeeda-meg-ac/source/corpus.clean.txt 10 data/EN/wiki_quality.txt 1
===Compilation===
===Tokenization===
Current step: Tokenizing input file...
real	0m12.787s
user	0m19.848s
sys	0m2.588s
Detected Language: EN
Current step: Tokenizing wikipedia phrases...
No provided expert labels.
===Part-Of-Speech Tagging===
===AutoPhrasing===
=== Current Settings ===
Iterations = 2
Minimum Support Threshold = 10
Maximum Length Threshold = 6
POS-Tagging Mode Disabled
Discard Ratio = 0.050000
Number of threads = 1
Labeling Method = DPDN
	Auto labels from knowledge bases
	Max Positive Samples = -1
Loading data.

In [12]:
# copy these results to sample-meg-pt
# !cp -r ../../data/$data_ac ../../data/$data_pt

## Corpus with company names

In [13]:
# dataset_path = '/home/ubuntu/users/nikita/data/indeed/indeedQA/question_answers.csv'
# company_path = '/home/ubuntu/users/nikita/data/indeed/indeedQA/fccid-companyName.csv'
# entity_emb_num_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/BERTembednum+seeds.txt')
# out_corpus_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/sentences_with_company.json')

In [15]:
%cd $base_dir/src/concept_learning/

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/concept_learning


In [16]:
# Use script
!python build_corpus_with_companies.py \
-d $base_dir/data/$data_ac/intermediate \
-b $base_dir/data/indeed-benchmark \
-rd /home/ubuntu/users/nikita/data/indeed/indeedQA


Processing lines: 100%|████████████████| 307122/307122 [11:54<00:00, 430.02it/s]


# Generate Embeddings

In [17]:
# details here: https://github.com/rit-git/meg-kb/tree/main/src/concept_learning

In [18]:
#change to concept learning directory
%cd $base_dir/src/concept_learning/

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/concept_learning


## Sentence Embedding

In [19]:
!python compute_keyphrase_embeddings.py -m bert-base-uncased -et ac -d ../../data/$data_ac/intermediate -c 750

computing entity-wise embedding: 100%|██████| 8028/8028 [07:01<00:00, 19.03it/s]
Saving embedding


## Concatenated Token Embedding

In [20]:
!python compute_keyphrase_embeddings.py -m bert-base-uncased -et pt -d ../../data/$data_pt/intermediate -c 750

computing entity-wise embedding: 100%|██████| 7973/7973 [07:15<00:00, 18.32it/s]
Saving embedding


## Token Embedding

In [21]:
# change directory to autophrase
%cd $base_dir/src/tools/AutoPhrase

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/tools/AutoPhrase


In [22]:
data_corel = 'sample-indeeda-corel'

In [23]:
!CUDA_VISIBLE_DEVICES=0 python extractBertEmbedding.py ../../../data/$data_corel/intermediate/ $thread

1
2021-07-07 00:47:48,693 : INFO : loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/ubuntu/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2021-07-07 00:47:49,035 : INFO : loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/ubuntu/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
2021-07-07 00:47:49,036 : INFO : Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embedding

## Add embeddings for seed instances

In [24]:
# corpus_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/sentences.json')
# seed_aligned_concepts_path = os.path.join(base_dir, f'data/indeed-benchmark/seed_aligned_concepts.csv')

# orig_bert_emb_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/BERTembed.txt')
# orig_bert_emb_num_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/BERTembednum.txt')

# new_bert_emb_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/BERTembed+seeds.txt')
# new_bert_emb_num_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/BERTembednum+seeds.txt')

In [26]:
%cd $base_dir/src/concept_learning/

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/concept_learning


In [27]:
# Using script

!python add_seed_instances_embeddings.py -m bert-base-uncased -et ac -d $base_dir/data/$data_ac/intermediate -b $base_dir/data/indeed-benchmark -c 750


Seed instances: ['walmart', 'amazon', 'subway', 'microsoft', 'target', 'business casual', 'uniform', 'hair color', 'tattoos', 'facial hair', 'shoes', 'piercings', 'delivery driver', 'store manager', 'cashier', 'package handler', 'sales associate', 'barista', 'dishwasher', 'weekly', 'biweekly', 'friday', 'saturday', 'health insurance', 'flexible schedule', '401k', 'paid vacation', 'sick leave', 'vision insurance', 'base pay', 'stock options', 'benefits', 'overtime pay', 'bonus', 'checks', 'direct deposit', 'prepaid card', 'drug test', 'criminal background check', 'employment verification', 'felons', 'criminals', 'disabled', 'drug addicts', 'high schoolers', 'misdemeanor', 'pregnant', 'students', 'seniors', 'hiring age', 'bachelors degree', 'prior experience', 'working permit', 'heavy lifting', 'night shift', 'dinner shift', 'early morning shift', '8 hour shift', 'christmas eve', 'early morning', 'hoilday', '7 days', 'saturday', 'sunday', 'weekend', 'full time', 'part time', 'seasonal', 

# Expand Seed Entities (clustering)

In [28]:
# details here: https://github.com/rit-git/meg-kb/tree/main/src/concept_learning

In [29]:
#change to concept learning directory
%cd $base_dir/src/concept_learning/

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/concept_learning


## knn sentence-embedding

In [30]:
clusters = 100
output = '../../data/'+data_ac+'/intermediate/knn_'+str(clusters)+'.csv'
dim = 768

In [31]:
!python compute_concept_clusters.py -d ../../data/$data_ac/intermediate/ -ca knn -s $clusters -dim $dim -o $output

finding nearest neighbors by entity: 100%|█| 8028/8028 [00:17<00:00, 449.07it/s]


## Analyzing Clustering Results

In [36]:
#Visit here: /meg_shared_scripts/meg-kb/src/analysis/concept_learning-test.ipynb

## EE-emb: concept embedding knn
(using all seed instances of a concept to find neighbors)

In [37]:
# bert_emb_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/BERTembed+seeds.txt')
# seed_concepts_path = os.path.join(base_dir, f'data/indeed-benchmark/seed_concepts.csv')
# seed_relations_path = os.path.join(base_dir, f'data/indeed-benchmark/seed_relations.csv')
# concept_knn_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/concept_knn_{cluster_size}.csv')

In [38]:
# Use script
# cluster_size = 100
!python compute_concept_seeds_knn.py \
-d $base_dir/data/$data_ac/intermediate \
-b $base_dir/data/indeed-benchmark \
-s 100 \
--kdt \
-o $base_dir/data/$data_ac/intermediate/concept_knn_100.csv

building entity index: 100%|██████████████| 8064/8064 [00:01<00:00, 6412.04it/s]
finding nearest neighbors by concept: 14it [00:00, 334.07it/s]


## EE-emb-contrastive

In [ ]:
# Use script
# cluster_size = None

!python compute_concept_contrastive_knn.py \
-d $base_dir/data/$data_ac/intermediate \
-b $base_dir/data/indeed-benchmark \
-s None \
-o $base_dir/data/$data_ac/intermediate/ee_concept_contr_knn_k=None.csv

## EE-LM-probe

In [ ]:
# Use scripts 
# topk = None

!python compute_EE_LM_probe.py \
-d $base_dir/data/$data_ac/intermediate \
-b $base_dir/data/indeed-benchmark \
-lm bert \
-topk None \
-o $base_dir/data/$data_ac/intermediate/ee_LM_bert_k=None.csv


## Entity expansion evaluation
Now using benchmark entities, mean reciprocal rank / recall @k

In [39]:
# seed_aligned_concepts_path = os.path.join(base_dir, f'data/indeed-benchmark/seed_aligned_concepts.csv')
# seed_aligned_relations_path = os.path.join(base_dir, f'data/indeed-benchmark/seed_aligned_relations_nodup.csv')
# benchmark_path = os.path.join(base_dir, f'data/indeed-benchmark/benchmark_evidence_clean.csv')
# concept_knn_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/concept_knn_1000.csv')

In [41]:
# Use script
!python eval_entities.py \
-b $base_dir/data/indeed-benchmark \
-pred $base_dir/data/$data_ac/intermediate/concept_knn_100.csv

Concept: company / company
seeds: ['walmart', 'amazon', 'subway', 'microsoft', 'target']
{
    "united states postal service": NaN,
    "wendys": NaN,
    "whataburger": NaN,
    "marriott international, inc.": NaN,
    "geico": 30,
    "foot locker": NaN,
    "the wendy's company": NaN,
    "mcdonald": NaN,
    "home depot": 5,
    "allied universal security services, systems and solutions": NaN,
    "goodwill industries": NaN,
    "american eagle outfitters": NaN,
    "chipotle mexican grill": NaN,
    "chick-fil-a": NaN,
    "heb": NaN,
    "sonic drive-in": NaN,
    "frito": NaN,
    "jcpenney": 21,
    "aldi": NaN,
    "electric": NaN,
    "pepsi": 39,
    "sitel": 42,
    "cvs health": NaN,
    "dick's sporting goods": NaN,
    "costco wholesale": NaN,
    "burlington stores": NaN,
    "dd": 84,
    "olive garden": 12,
    "fedex": 36,
    "dunkin donuts": 43,
    "primark": NaN,
    "best buy": 13,
    "frito lay": 33,
    "marshalls": 35,
    "domino's": NaN,
    "tim hortons":

# Relation Extraction Baselines
Currently only for single relation. TODO: include all relations

## Null baseline - Cartesian product

In [226]:
# seed_concepts_path = os.path.join(base_dir, f'data/indeed-benchmark/seed_concepts.csv')
# seed_relations_path = os.path.join(base_dir, f'data/indeed-benchmark/seed_relations.csv')
# seed_aligned_concepts_path = os.path.join(base_dir, f'data/indeed-benchmark/seed_aligned_concepts.csv')
# seed_aligned_relations_path = os.path.join(base_dir, f'data/indeed-benchmark/seed_aligned_relations_nodup.csv')
# # knn_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/knn_{cluster_size}.csv')
# concept_knn_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/concept_knn_1000.csv')

# relation = 'has_benefits'
# cartesian_re_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/rel_cartesian-{relation}.csv')

In [93]:
relations = ['has_benefits', 'has_dress_code', 'has_job_position', 'has_background_screening', 'has_pay_schedule', 'has_shifts']

for relation in relations:
    !python relation_extraction_cartesian.py \
    -d $base_dir/data/$data_ac/intermediate \
    -b $base_dir/data/indeed-benchmark \
    -o $base_dir/data/$data_ac/intermediate/rel_extraction-$relation-EE=emb-RE=Ct.csv \
    -r $relation \
    -cknn $base_dir/data/$data_ac/intermediate/ee_concept_knn_k=None.csv \
    -rank sim \
    -rev \
    -topk 100

company 	 benefits
seed_heads: ['walmart', 'amazon', 'subway', 'microsoft', 'target']
seed_tails: ['health insurance', 'flexible schedule', '401k', 'paid vacation', 'sick leave', 'vision insurance']
heads: ('walmart', 'amazon', 'subway', 'microsoft', 'target', 'wal mart', 'costco', 'dollar general', 'apple', 'home depot', 'walgreens', 'kroger', 'publix', 'pizza hut', 'starbucks', 'panera bread', 'olive garden', 'best buy', 'mcdonalds', 'family dollar', 'old navy', 'dollar tree', 'spectrum', 'kfc', 'cracker barrel', 'jcpenney', 'safeway', 'whole foods', 'menards', 'cvs', 'ihop', 'center', 'burlington', "sam 's club", 'geico', 'usps', 'burger king', 'frito lay', 'chipotle', 'marshalls', 'fedex', 'taco bell', 'verizon', 'pepsi', 'petsmart', 'at&t', 'sitel', 'dunkin donuts', 'g4s', 'company', 'warehouse', 'jcp', 'retail', 'tj maxx', 'wells fargo', 'pharmacy', 'post office', 'pepsico', 'management', "macy 's", 't mobile', 'chick fil a', 'call center', 'union', 'cashier', 'department', 'home

company 	 background_screening
seed_heads: ['walmart', 'amazon', 'subway', 'microsoft', 'target']
seed_tails: ['drug test', 'criminal background check', 'employment verification']
heads: ('walmart', 'amazon', 'subway', 'microsoft', 'target', 'wal mart', 'costco', 'dollar general', 'apple', 'home depot', 'walgreens', 'kroger', 'publix', 'pizza hut', 'starbucks', 'panera bread', 'olive garden', 'best buy', 'mcdonalds', 'family dollar', 'old navy', 'dollar tree', 'spectrum', 'kfc', 'cracker barrel', 'jcpenney', 'safeway', 'whole foods', 'menards', 'cvs', 'ihop', 'center', 'burlington', "sam 's club", 'geico', 'usps', 'burger king', 'frito lay', 'chipotle', 'marshalls', 'fedex', 'taco bell', 'verizon', 'pepsi', 'petsmart', 'at&t', 'sitel', 'dunkin donuts', 'g4s', 'company', 'warehouse', 'jcp', 'retail', 'tj maxx', 'wells fargo', 'pharmacy', 'post office', 'pepsico', 'management', "macy 's", 't mobile', 'chick fil a', 'call center', 'union', 'cashier', 'department', 'home office', 'asset pr

In [94]:
for relation in relations:
    !python relation_extraction_cartesian.py \
    -d $base_dir/data/$data_ac/intermediate \
    -b $base_dir/data/indeed-benchmark \
    -o $base_dir/data/$data_ac/intermediate/rel_extraction-$relation-EE=emb_contr-RE=Ct.csv \
    -r $relation \
    -cknn $base_dir/data/$data_ac/intermediate/ee_concept_contr_knn_k=None.csv \
    -rank sim+margin \
    -rev \
    -topk 100

company 	 benefits
seed_heads: ['walmart', 'amazon', 'subway', 'microsoft', 'target']
seed_tails: ['health insurance', 'flexible schedule', '401k', 'paid vacation', 'sick leave', 'vision insurance']
heads: ('walmart', 'amazon', 'subway', 'microsoft', 'target', 'apple', 'costco', 'wal mart', 'amazon', 'walmart', 'home depot', 'dollar general', 'jcpenney', 'starbucks', 'best buy', 'walgreens', 'publix', 'geico', 'verizon', 'sitel', 'kroger', 'panera bread', 'spectrum', 'cracker barrel', 'whole foods', 'g4s', 'frito lay', 'old navy', 'family dollar', 'jcp', 'at&t', 'pizza hut', 'target', 'olive garden', 'menards', 'tj maxx', 't mobile', "sam 's club", 'kfc', 'usps', 'safeway', 'planet fitness', 'dollar tree', 'cvs', 'wells fargo', 'burlington', 'subway', 'pepsico', 'dunkin donuts', 'pepsi', 'mcdonalds', 'marshalls', 'chipotle', 'center', 'burger king', 'fedex', 'ihop', 'america', 'home office', 'taco bell', 'petsmart', 'company', 'chick fil a', 'dd', 'india', 'hobby lobby', "macy 's", 'co

company 	 background_screening
seed_heads: ['walmart', 'amazon', 'subway', 'microsoft', 'target']
seed_tails: ['drug test', 'criminal background check', 'employment verification']
heads: ('walmart', 'amazon', 'subway', 'microsoft', 'target', 'apple', 'costco', 'wal mart', 'amazon', 'walmart', 'home depot', 'dollar general', 'jcpenney', 'starbucks', 'best buy', 'walgreens', 'publix', 'geico', 'verizon', 'sitel', 'kroger', 'panera bread', 'spectrum', 'cracker barrel', 'whole foods', 'g4s', 'frito lay', 'old navy', 'family dollar', 'jcp', 'at&t', 'pizza hut', 'target', 'olive garden', 'menards', 'tj maxx', 't mobile', "sam 's club", 'kfc', 'usps', 'safeway', 'planet fitness', 'dollar tree', 'cvs', 'wells fargo', 'burlington', 'subway', 'pepsico', 'dunkin donuts', 'pepsi', 'mcdonalds', 'marshalls', 'chipotle', 'center', 'burger king', 'fedex', 'ihop', 'america', 'home office', 'taco bell', 'petsmart', 'company', 'chick fil a', 'dd', 'india', 'hobby lobby', "macy 's", 'corporate office', 'd

In [95]:
for relation in relations:
    !python relation_extraction_cartesian.py \
    -d $base_dir/data/$data_ac/intermediate \
    -b $base_dir/data/indeed-benchmark \
    -o $base_dir/data/$data_ac/intermediate/rel_extraction-$relation-EE=LM_bert-RE=Ct.csv \
    -r $relation \
    -cknn $base_dir/data/$data_ac/intermediate/ee_LM_bert_k=None.csv \
    -rank lm_score \
    -rev \
    -topk 100

company 	 benefits
seed_heads: ['walmart', 'amazon', 'subway', 'microsoft', 'target']
seed_tails: ['health insurance', 'flexible schedule', '401k', 'paid vacation', 'sick leave', 'vision insurance']
heads: ('walmart', 'amazon', 'subway', 'microsoft', 'target', 'nike', 'apple', 'ibm', 'sears', 'starbucks', 'target corporation', 'google', 'intel', 'pepsi', 'samsung', 'facebook', 'dell', 'microsoft office', 'bmw', 'hp', 'apple store', 'yahoo', 'nissan', 'a & w', 'lego', 'boeing', 'toyota', 'dupont', 'disney', 'ge', 'ford', 'adobe', 'company / store', 'united healthcare', 'pepsi cola', 'canon', 'cadillac', 'gap inc', 'c & w', 'corporate / office', 'insurance company', 'liberty', 'amazons', 'microsoft word', 'india office', 'apple pay', 'pepsico', 'public company', 'apple watch', 'nintendo', 'big mac', 'american eagle', 'honda', 'twitter', 'fox', 'h & m', 'youtube', 'at & t', 'd & d', 'smart tv', 'big box stores', 'hyundai', 'henry ford', 'ad & d', 'general warehouse', 'marvel', 'google dri

company 	 background_screening
seed_heads: ['walmart', 'amazon', 'subway', 'microsoft', 'target']
seed_tails: ['drug test', 'criminal background check', 'employment verification']
heads: ('walmart', 'amazon', 'subway', 'microsoft', 'target', 'nike', 'apple', 'ibm', 'sears', 'starbucks', 'target corporation', 'google', 'intel', 'pepsi', 'samsung', 'facebook', 'dell', 'microsoft office', 'bmw', 'hp', 'apple store', 'yahoo', 'nissan', 'a & w', 'lego', 'boeing', 'toyota', 'dupont', 'disney', 'ge', 'ford', 'adobe', 'company / store', 'united healthcare', 'pepsi cola', 'canon', 'cadillac', 'gap inc', 'c & w', 'corporate / office', 'insurance company', 'liberty', 'amazons', 'microsoft word', 'india office', 'apple pay', 'pepsico', 'public company', 'apple watch', 'nintendo', 'big mac', 'american eagle', 'honda', 'twitter', 'fox', 'h & m', 'youtube', 'at & t', 'd & d', 'smart tv', 'big box stores', 'hyundai', 'henry ford', 'ad & d', 'general warehouse', 'marvel', 'google drive', 'first advanta

## RE - weighted average (X)

In [ ]:
# seed_aligned_concepts_path = os.path.join(base_dir, f'data/indeed-benchmark/seed_aligned_concepts.csv')
# seed_aligned_relations_path = os.path.join(base_dir, f'data/indeed-benchmark/seed_aligned_relations_nodup.csv')
# # knn_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/knn_{cluster_size}.csv')
# concept_knn_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/concept_knn_100.csv')
# bert_emb_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/BERTembed+seeds.txt')

# templates_path = 'templates_manual.json'

# extraction_save_path = os.path.join(base_dir, f'data/{data_ac}/intermediate/rel_extraction.csv')

In [45]:
# Use script 
!python relation_extraction_avg_scores.py \
-d $base_dir/data/$data_ac/intermediate \
-b $base_dir/data/indeed-benchmark \
-o $base_dir/data/$data_ac/intermediate/rel_extraction-has_benefits-RE=avg.csv \
-r has_benefits \
-cknn $base_dir/data/$data_ac/intermediate/concept_knn_100.csv \
-dim 768
# -topk 300


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
company 	 benefits
seed_heads: ['walmart', 'amazon', 'subway', 'microsoft', 'target']
seed_tails: ['health insurance', 'flexible schedule', '401k', 'paid vacation', 'sick leave', 'vision insurance']
100%|█████████████████████████████████████████| 102/102 [00:37<00:00,  2.71it/s]


In [46]:
# Use script 
!python relation_extraction_avg_scores.py \
-d $base_dir/data/$data_ac/intermediate \
-b $base_dir/data/indeed-benchmark \
-o $base_dir/data/$data_ac/intermediate/rel_extraction-has_dress_code-RE=avg.csv \
-r has_dress_code \
-cknn $base_dir/data/$data_ac/intermediate/concept_knn_100.csv \
-dim 768
# -topk 300


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
company 	 dress_code
seed_heads: ['walmart', 'amazon', 'subway', 'microsoft', 'target']
seed_tails: ['business casual', 'uniform', 'hair color', 'tattoos', 'facial hair', 'shoes', 'piercings']
100%|█████████████████████████████████████████| 102/102 [00:30<00:00,  3.31it/s]


# Knowledge Verification baseline
(finding co-occurrences of head / tail from corpus)

## has_dress_code

In [ ]:
!python knowledge_verification_entail.py \
-d $base_dir/data/$data_ac/intermediate \
-in $base_dir/data/$data_ac/intermediate/rel_extraction-has_dress_code-EE=LM_bert-RE=Ct.csv \
-o_kv $base_dir/data/$data_ac/intermediate/kv_evidences-has_dress_code-EE=LM_bert-RE=Ct.json \
-o_re $base_dir/data/$data_ac/intermediate/rel_extraction-has_dress_code-EE=LM_bert-RE=Ct+KV=0.9.csv \
-r $yutong_base_dir/models/roberta-large \
-rs $yutong_base_dir/repos/Roberta_SES/checkpoints/epoch=2-valid_loss=-0.2620-valid_acc_end=0.9223.ckpt \
-p_kv 0.7 \
-p_re 0.9 \
--fast_skip 3

In [ ]:
# !python knowledge_verification_entail.py \
# -d $base_dir/data/$data_ac/intermediate \
# -in $base_dir/data/$data_ac/intermediate/rel_extraction-has_dress_code-RE=avg.csv \
# -o_kv $base_dir/data/$data_ac/intermediate/kv_evidences-has_dress_code-RE=avg.json \
# -o_re $base_dir/data/$data_ac/intermediate/rel_extraction-has_dress_code-RE=avg+KV=0.9.csv \
# -r $yutong_base_dir/models/roberta-large \
# -rs $yutong_base_dir/repos/Roberta_SES/checkpoints/epoch=2-valid_loss=-0.2620-valid_acc_end=0.9223.ckpt \
# -p_kv 0.7 \
# -p_re 0.9 \
# --fast_skip

## has_benefits

In [48]:
!python knowledge_verification_entail.py \
-d $base_dir/data/$data_ac/intermediate \
-in $base_dir/data/$data_ac/intermediate/rel_extraction-has_benefits-EE=LM_bert-RE=Ct.csv \
-o_kv $base_dir/data/$data_ac/intermediate/kv_evidences-has_benefits-EE=LM_bert-RE=Ct.json \
-o_re $base_dir/data/$data_ac/intermediate/rel_extraction-has_benefits-EE=LM_bert-RE=Ct+KV=0.9.csv \
-r $yutong_base_dir/models/roberta-large \
-rs $yutong_base_dir/repos/Roberta_SES/checkpoints/epoch=2-valid_loss=-0.2620-valid_acc_end=0.9223.ckpt \
-p_kv 0.7 \
-p_re 0.9 \
--fast_skip 3

Loading files...
Finding evidence for rels: 100%|████████| 10404/10404 [2:20:15<00:00,  1.24it/s]


In [ ]:
# !python knowledge_verification_entail.py \
# -d $base_dir/data/$data_ac/intermediate \
# -in $base_dir/data/$data_ac/intermediate/rel_extraction-has_benefits-RE=avg.csv \
# -o_kv $base_dir/data/$data_ac/intermediate/kv_evidences-has_benefits-RE=avg.json \
# -o_re $base_dir/data/$data_ac/intermediate/rel_extraction-has_benefits-RE=avg+KV=0.9.csv \
# -r $yutong_base_dir/models/roberta-large \
# -rs $yutong_base_dir/repos/Roberta_SES/checkpoints/epoch=2-valid_loss=-0.2620-valid_acc_end=0.9223.ckpt \
# -p_kv 0.7 \
# -p_re 0.9 \
# --fast_skip

## has_pay_schedule

In [76]:
!python knowledge_verification_entail.py \
-d $base_dir/data/$data_ac/intermediate \
-in $base_dir/data/$data_ac/intermediate/rel_extraction-has_pay_schedule-EE=LM_bert-RE=Ct.csv \
-o_kv $base_dir/data/$data_ac/intermediate/kv_evidences-has_pay_schedule-EE=LM_bert-RE=Ct.json \
-o_re $base_dir/data/$data_ac/intermediate/rel_extraction-has_pay_schedule-EE=LM_bert-RE=Ct+KV=0.9.csv \
-r $yutong_base_dir/models/roberta-large \
-rs $yutong_base_dir/repos/Roberta_SES/checkpoints/epoch=2-valid_loss=-0.2620-valid_acc_end=0.9223.ckpt \
-p_kv 0.7 \
-p_re 0.9 \
--fast_skip 3

Loading files...
Finding evidence for rels: 100%|████████| 10302/10302 [1:33:26<00:00,  1.84it/s]


## has_background_screening

In [78]:
!python knowledge_verification_entail.py \
-d $base_dir/data/$data_ac/intermediate \
-in $base_dir/data/$data_ac/intermediate/rel_extraction-has_background_screening-EE=LM_bert-RE=Ct.csv \
-o_kv $base_dir/data/$data_ac/intermediate/kv_evidences-has_background_screening-EE=LM_bert-RE=Ct.json \
-o_re $base_dir/data/$data_ac/intermediate/rel_extraction-has_background_screening-EE=LM_bert-RE=Ct+KV=0.9.csv \
-r $yutong_base_dir/models/roberta-large \
-rs $yutong_base_dir/repos/Roberta_SES/checkpoints/epoch=2-valid_loss=-0.2620-valid_acc_end=0.9223.ckpt \
-p_kv 0.7 \
-p_re 0.9 \
--fast_skip 3

Loading files...
Finding evidence for rels: 100%|████████| 10302/10302 [7:21:16<00:00,  2.57s/it]


# Full Pipeline Evaluation (on relations)

## has_dress_code

In [96]:
!python eval_relations.py \
-b $base_dir/data/indeed-benchmark \
-pred $base_dir/data/$data_ac/intermediate/rel_extraction-has_dress_code-EE=emb-RE=Ct.csv \
-r has_dress_code

--- RE Results ---
Benchmark relations: 107
Predicted relations: 10000
Intersection: 15
P = 0.0015, R = 0.1402, F1 = 0.0030

Intersection:
('at&t', 'has_dress_code', 'uniform')
('best buy', 'has_dress_code', 'uniform')
('costco', 'has_dress_code', 'hair color')
('dd', 'has_dress_code', 'facial hair')
('dollar general', 'has_dress_code', 'strict dress code')
('dollar tree', 'has_dress_code', 'professional')
('dollar tree', 'has_dress_code', 'uniform')
('family dollar', 'has_dress_code', 'facial hair')
('jcpenney', 'has_dress_code', 'uniform policy')
('marshalls', 'has_dress_code', 'color hair')
('olive garden', 'has_dress_code', 'facial hair')
('subway', 'has_dress_code', 'piercings')
('taco bell', 'has_dress_code', 'nose rings')
('walgreens', 'has_dress_code', 'hair color')
('walmart', 'has_dress_code', 'uniform')



In [ ]:
!python eval_relations.py \
-b $base_dir/data/indeed-benchmark \
-pred $base_dir/data/$data_ac/intermediate/rel_extraction-has_dress_code-RE=Ct+KV=0.9.csv \
-r has_dress_code

In [97]:
!python eval_relations.py \
-b $base_dir/data/indeed-benchmark \
-pred $base_dir/data/$data_ac/intermediate/rel_extraction-has_dress_code-EE=emb_contr-RE=Ct.csv \
-r has_dress_code

--- RE Results ---
Benchmark relations: 107
Predicted relations: 8928
Intersection: 15
P = 0.0017, R = 0.1402, F1 = 0.0033

Intersection:
('at&t', 'has_dress_code', 'uniform')
('best buy', 'has_dress_code', 'uniform')
('costco', 'has_dress_code', 'hair color')
('cracker barrel', 'has_dress_code', 'dress pants')
('dd', 'has_dress_code', 'facial hair')
('dollar tree', 'has_dress_code', 'uniform')
('family dollar', 'has_dress_code', 'facial hair')
('olive garden', 'has_dress_code', 'facial hair')
('subway', 'has_dress_code', 'piercings')
('taco bell', 'has_dress_code', 'jeans')
('taco bell', 'has_dress_code', 'nose rings')
('walgreens', 'has_dress_code', 'hair color')
('walmart', 'has_dress_code', 'black jeans')
('walmart', 'has_dress_code', 'jeans')
('walmart', 'has_dress_code', 'uniform')



In [98]:
!python eval_relations.py \
-b $base_dir/data/indeed-benchmark \
-pred $base_dir/data/$data_ac/intermediate/rel_extraction-has_dress_code-EE=LM_bert-RE=Ct.csv \
-r has_dress_code

--- RE Results ---
Benchmark relations: 107
Predicted relations: 10000
Intersection: 6
P = 0.0006, R = 0.0561, F1 = 0.0012

Intersection:
('subway', 'has_dress_code', 'piercings')
('walmart', 'has_dress_code', 'black jeans')
('walmart', 'has_dress_code', 'blue collar')
('walmart', 'has_dress_code', 'face tattoos')
('walmart', 'has_dress_code', 'jeans')
('walmart', 'has_dress_code', 'uniform')



## has_benefits

In [99]:
!python eval_relations.py \
-b $base_dir/data/indeed-benchmark \
-pred $base_dir/data/$data_ac/intermediate/rel_extraction-has_benefits-EE=emb-RE=Ct.csv \
-r has_benefits

--- RE Results ---
Benchmark relations: 56
Predicted relations: 10000
Intersection: 9
P = 0.0009, R = 0.1607, F1 = 0.0018

Intersection:
('burger king', 'has_benefits', 'health')
('burger king', 'has_benefits', 'sick leave')
('dollar general', 'has_benefits', 'health insurance')
('g4s', 'has_benefits', 'sick leave')
('starbucks', 'has_benefits', 'health')
('target', 'has_benefits', 'health insurance')
('walmart', 'has_benefits', '401k')
('walmart', 'has_benefits', 'life insurance')
('walmart', 'has_benefits', 'paid vacations')



In [ ]:
!python eval_relations.py \
-b $base_dir/data/indeed-benchmark \
-pred $base_dir/data/$data_ac/intermediate/rel_extraction-has_benefits-RE=Ct+KV=0.9.csv \
-r has_benefits

In [100]:
!python eval_relations.py \
-b $base_dir/data/indeed-benchmark \
-pred $base_dir/data/$data_ac/intermediate/rel_extraction-has_benefits-EE=emb_contr-RE=Ct.csv \
-r has_benefits

--- RE Results ---
Benchmark relations: 56
Predicted relations: 9120
Intersection: 9
P = 0.0010, R = 0.1607, F1 = 0.0020

Intersection:
('burger king', 'has_benefits', 'health')
('burger king', 'has_benefits', 'sick leave')
('dollar general', 'has_benefits', 'health insurance')
('g4s', 'has_benefits', 'sick leave')
('starbucks', 'has_benefits', 'health')
('target', 'has_benefits', 'health insurance')
('walmart', 'has_benefits', '401k')
('walmart', 'has_benefits', 'life insurance')
('walmart', 'has_benefits', 'paid vacations')



In [101]:
!python eval_relations.py \
-b $base_dir/data/indeed-benchmark \
-pred $base_dir/data/$data_ac/intermediate/rel_extraction-has_benefits-EE=LM_bert-RE=Ct.csv \
-r has_benefits

--- RE Results ---
Benchmark relations: 56
Predicted relations: 10000
Intersection: 6
P = 0.0006, R = 0.1071, F1 = 0.0012

Intersection:
('starbucks', 'has_benefits', 'health')
('target', 'has_benefits', 'health insurance')
('walmart', 'has_benefits', '401k')
('walmart', 'has_benefits', 'health benefits')
('walmart', 'has_benefits', 'life insurance')
('walmart', 'has_benefits', 'paid vacations')



## has_background_screening

In [102]:
!python eval_relations.py \
-b $base_dir/data/indeed-benchmark \
-pred $base_dir/data/$data_ac/intermediate/rel_extraction-has_background_screening-EE=emb-RE=Ct.csv \
-r has_background_screening

--- RE Results ---
Benchmark relations: 207
Predicted relations: 10000
Intersection: 42
P = 0.0042, R = 0.2029, F1 = 0.0082

Intersection:
('burger king', 'has_background_screening', 'credit history')
('burger king', 'has_background_screening', 'drug test')
('burger king', 'has_background_screening', 'random drug tests')
('cracker barrel', 'has_background_screening', 'credit history')
('cracker barrel', 'has_background_screening', 'criminal background')
('cracker barrel', 'has_background_screening', 'criminal background check')
('dollar general', 'has_background_screening', 'driving record')
('dollar general', 'has_background_screening', 'social security number')
('family dollar', 'has_background_screening', 'credit history')
('family dollar', 'has_background_screening', 'criminal record')
('fedex', 'has_background_screening', 'random drug test')
('geico', 'has_background_screening', 'background check')
('geico', 'has_background_screening', 'drug test')
('marshalls', 'has_background_sc

In [103]:
!python eval_relations.py \
-b $base_dir/data/indeed-benchmark \
-pred $base_dir/data/$data_ac/intermediate/rel_extraction-has_background_screening-EE=emb_contr-RE=Ct.csv \
-r has_background_screening

--- RE Results ---
Benchmark relations: 207
Predicted relations: 9408
Intersection: 22
P = 0.0023, R = 0.1063, F1 = 0.0046

Intersection:
('burger king', 'has_background_screening', 'credit history')
('burger king', 'has_background_screening', 'drug test')
('cracker barrel', 'has_background_screening', 'credit history')
('cracker barrel', 'has_background_screening', 'criminal background')
('cracker barrel', 'has_background_screening', 'criminal background check')
('dollar general', 'has_background_screening', 'driving record')
('dollar general', 'has_background_screening', 'social security number')
('family dollar', 'has_background_screening', 'credit history')
('geico', 'has_background_screening', 'drug test')
('old navy', 'has_background_screening', 'criminal background check')
('target', 'has_background_screening', 'drug test')
('target', 'has_background_screening', 'previous employment')
('verizon', 'has_background_screening', 'criminal background')
('walgreens', 'has_background_sc

In [104]:
!python eval_relations.py \
-b $base_dir/data/indeed-benchmark \
-pred $base_dir/data/$data_ac/intermediate/rel_extraction-has_background_screening-EE=LM_bert-RE=Ct.csv \
-r has_background_screening

--- RE Results ---
Benchmark relations: 207
Predicted relations: 10000
Intersection: 22
P = 0.0022, R = 0.1063, F1 = 0.0043

Intersection:
('amazon', 'has_background_screening', 'random drug testing')
('geico', 'has_background_screening', 'background check')
('geico', 'has_background_screening', 'drug test')
('subway', 'has_background_screening', 'drug testing')
('subway', 'has_background_screening', 'drug tests')
('subway', 'has_background_screening', 'urine drug test')
('target', 'has_background_screening', 'background check')
('target', 'has_background_screening', 'criminal record')
('target', 'has_background_screening', 'drug test')
('target', 'has_background_screening', 'drug tests')
('target', 'has_background_screening', 'random drug testing')
('target', 'has_background_screening', 'urine test')
('walmart', 'has_background_screening', 'background check')
('walmart', 'has_background_screening', 'blood test')
('walmart', 'has_background_screening', 'credit check')
('walmart', 'has_

In [ ]:
# Discussions:
# coherence clustering / ensemble models?
# trying for other relations or entities
# using entities in sub-categories
# fine-tuning
# ambiguous samples (high for pos and neg)
# quantitative-evaluation

# Mine Prompts

In [108]:
# Explore various techniques
# Get prompts "between" entities
# Get prompts by syntactic parsing
# Get prompts by paraphrasing
# Get prompts uisng AutoPrompt

In [109]:
# visit here: /meg-kb/src/analysis/pattern_mining.ipynb

# Retrieve Prompt Evidence

In [109]:
# visit here: /meg-kb/src/analysis/lm_probing.ipynb

# Suggest Quality Prompts